# Exploratory notebook 
The goal of this notebook is to conduct the research part of "predicting the results of a match". Importing data from past seasons, transforming that data to extract new indicators (prétraitement et EDA), developping and evaluating models.

In [1]:

import pandas as pd             # manipulation de données tabulaires
import sys
import os
import json                     # pour afficher proprement le JSON si besoin
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(".."))


In [2]:
from src.etl import fetch_matches, save_raw_matches

In [3]:
matches_json=fetch_matches(2015,2024) #Ligue 1 saison 2024-2025

save_raw_matches(matches_json)

'data/raw/FL1_2024.json'

In [4]:
matches_list = matches_json["matches"]
df_matches = pd.json_normalize(matches_list)

In [10]:
df_matches.head()

,id,utcDate,status,matchday,stage,group,lastUpdated,referees,area.id,area.name,...,awayTeam.shortName,awayTeam.tla,awayTeam.crest,score.winner,score.duration,score.fullTime.home,score.fullTime.away,score.halfTime.home,score.halfTime.away,odds.msg
0,497958,2024-08-16T18:45:00Z,FINISHED,1,REGULAR_SEASON,None,2025-06-01T20:20:47Z,"[{'id': 57042, 'name': 'Willy Delajod', 'type'...",2081,France,...,PSG,PSG,https://crests.football-data.org/524.png,AWAY_TEAM,REGULAR,1,4,0,1,Activate Odds-Package in User-Panel to retriev...
1,497951,2024-08-17T15:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-06-01T20:20:47Z,"[{'id': 43926, 'name': 'Benoît Bastien', 'type...",2081,France,...,Marseille,MAR,https://crests.football-data.org/516.png,AWAY_TEAM,REGULAR,1,5,1,3,Activate Odds-Package in User-Panel to retriev...
2,497956,2024-08-17T17:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-06-01T20:20:47Z,"[{'id': 57092, 'name': 'Eric Wattellier', 'typ...",2081,France,...,Lille,LIL,https://crests.football-data.org/521.png,AWAY_TEAM,REGULAR,0,2,0,1,Activate Odds-Package in User-Panel to retriev...
3,497955,2024-08-17T19:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-06-01T20:20:47Z,"[{'id': 9374, 'name': 'Clément Turpin', 'type'...",2081,France,...,Saint-Étienne,ASS,https://crests.football-data.org/527.png,HOME_TEAM,REGULAR,1,0,1,0,Activate Odds-Package in User-Panel to retriev...
4,497957,2024-08-18T13:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-06-01T20:20:47Z,"[{'id': 57087, 'name': 'Romain Lissorgue', 'ty...",2081,France,...,Nice,NIC,https://crests.football-data.org/522.png,HOME_TEAM,REGULAR,2,1,1,1,Activate Odds-Package in User-Panel to retriev...


In [11]:
df_matches.columns


Index(['id', 'utcDate', 'status', 'matchday', 'stage', 'group', 'lastUpdated',
       'referees', 'area.id', 'area.name', 'area.code', 'area.flag',
       'competition.id', 'competition.name', 'competition.code',
       'competition.type', 'competition.emblem', 'season.id',
       'season.startDate', 'season.endDate', 'season.currentMatchday',
       'season.winner', 'homeTeam.id', 'homeTeam.name', 'homeTeam.shortName',
       'homeTeam.tla', 'homeTeam.crest', 'awayTeam.id', 'awayTeam.name',
       'awayTeam.shortName', 'awayTeam.tla', 'awayTeam.crest', 'score.winner',
       'score.duration', 'score.fullTime.home', 'score.fullTime.away',
       'score.halfTime.home', 'score.halfTime.away', 'odds.msg'],
      dtype='object')

There are a lot of columns that we ca let go of: date, stage, group, lastUpdated, referees, all the things explaining what competition it is, quite a lot of the variables explaining the teams (crest, shortname, tla as they are redundant.)

In [14]:
df_smaller=df_matches[["status","matchday","homeTeam.id","homeTeam.name","awayTeam.id","awayTeam.name",
                       "score.winner","score.fullTime.home","score.fullTime.away"]]

In [20]:
df_smaller.head()

,status,matchday,homeTeam.id,homeTeam.name,awayTeam.id,awayTeam.name,score.winner,score.fullTime.home,score.fullTime.away
0,FINISHED,1,533,Le Havre AC,524,Paris Saint-Germain FC,AWAY_TEAM,1,4
1,FINISHED,1,512,Stade Brestois 29,516,Olympique de Marseille,AWAY_TEAM,1,5
2,FINISHED,1,547,Stade de Reims,521,Lille OSC,AWAY_TEAM,0,2
3,FINISHED,1,548,AS Monaco FC,527,AS Saint-Étienne,HOME_TEAM,1,0
4,FINISHED,1,519,AJ Auxerre,522,OGC Nice,HOME_TEAM,2,1


This seems like a good base to make our predictions, and like we could construct some new variables from this. The variable e will want to predict is score.winner. The problem is we can't predict score winner knowing the number of goals scored... Of course. But we could create variables like "average number of goal scored per match earlier  in the season" and same for goals conceded. Similarly we could create variables such as "home team number of wins/loss at home" and "away team number of win/loss away" and such. I want to note that the variables related to the number of goals will need to be discarded for the "predicting the end of a season" project because we will predict HOME/DRAW/AWAY but not the exact score.

In [40]:
def prepa_donnee(df):
    df = df.sort_values(["matchday", "homeTeam.id"])

    df_home = df[["matchday", "homeTeam.id", 
                "score.fullTime.home", "score.fullTime.away", "score.winner"]].rename(
        columns={
            "homeTeam.id": "team_id",
            "score.fullTime.home": "goals_for",
            "score.fullTime.away": "goals_against"
        }
    )
    df_home["is_home"] = True

    df_away = df[["matchday", "awayTeam.id", 
                "score.fullTime.home", "score.fullTime.away", "score.winner"]].rename(
        columns={
            "awayTeam.id": "team_id",
            "score.fullTime.home": "goals_against",
            "score.fullTime.away": "goals_for"
        }
    )
    df_away["is_home"] = False

    df_home["win"]  = df_home["score.winner"] == "HOME_TEAM"
    df_home["loss"] = df_home["score.winner"] == "AWAY_TEAM"

    df_away["win"]  = df_away["score.winner"] == "AWAY_TEAM"
    df_away["loss"] = df_away["score.winner"] == "HOME_TEAM"

    df_long = pd.concat([df_home, df_away], ignore_index=True)
    df_long = df_long.sort_values(["team_id", "matchday"])

    df_long["points"] = df_long.apply(pts, axis=1)
    df_long["points_before_match"] = (
        df_long.groupby("team_id")["points"]
            .cumsum()
            .shift(1)
    )

    df_long["goal_diff"] = df_long["goals_for"] - df_long["goals_against"]


    df_long["avg_goals_for"] = df_long.groupby("team_id")["goals_for"] \
                                    .transform(lambda s: s.shift(1).expanding().mean())

    df_long["avg_goals_against"] = df_long.groupby("team_id")["goals_against"] \
                                        .transform(lambda s: s.shift(1).expanding().mean())

    df_long["cum_goals_for"] = (
        df_long.groupby("team_id")["goals_for"].cumsum().shift(1)
    )
    df_long["cum_goals_against"] = (
        df_long.groupby("team_id")["goals_against"].cumsum().shift(1)
    )
    df_long["cum_goal_diff"] = df_long["cum_goals_for"] - df_long["cum_goals_against"]


    df_long["home_win_rate"] = (
        df_long[df_long.is_home]
        .groupby("team_id")["win"]
        .transform(lambda s: s.shift(1).expanding().mean())
    )

    df_long["away_win_rate"] = (
        df_long[~df_long.is_home]
        .groupby("team_id")["win"]
        .transform(lambda s: s.shift(1).expanding().mean())
    )

    df_long["away_loss_rate"] = (
        df_long[~df_long.is_home]
        .groupby("team_id")["loss"]
        .transform(lambda s: s.shift(1).expanding().mean())
    )

    df_long["home_loss_rate"] = (
        df_long[df_long.is_home]
        .groupby("team_id")["loss"]
        .transform(lambda s: s.shift(1).expanding().mean())
    )

    df_long["ranking_before_match"] = (
        df_long.groupby("matchday")
        .apply(
            lambda g: g.sort_values(
                ["points_before_match", "cum_goal_diff", "cum_goals_for"],
                ascending=[False, False, False]
            ).assign(ranking=lambda x: range(1, len(x)+1)),
            include_groups=False          
        )
        .reset_index(drop=True)["ranking"]
    )


    df_long["form_last5"] = (
        df_long.groupby("team_id")["points"]
            .transform(lambda s: s.shift(1).rolling(5, min_periods=1).sum())
    )
    return df_long[["team_id", "matchday","points_before_match", "cum_goal_diff", "cum_goals_for", "cum_goals_against",
    "avg_goals_for", "avg_goals_against",
    "home_win_rate", "away_win_rate", "home_loss_rate", "away_loss_rate",
    "ranking_before_match", "form_last5"]]

In [41]:
df_stats=prepa_donnee(df_smaller)

In [59]:
df_stats.head()

,team_id,matchday,points_before_match,cum_goal_diff,cum_goals_for,cum_goals_against,avg_goals_for,avg_goals_against,home_win_rate,away_win_rate,home_loss_rate,away_loss_rate,ranking_before_match,form_last5
0,511,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
317,511,2,1.0,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,12,1.0
18,511,3,2.0,0.0,1.0,1.0,0.500000,0.500000,0.0,NaN,0.0,NaN,1,2.0
27,511,4,2.0,-2.0,2.0,4.0,0.666667,1.333333,0.0,NaN,0.5,NaN,10,2.0
342,511,5,5.0,0.0,4.0,4.0,1.000000,1.000000,NaN,0.0,NaN,0.0,1,5.0


In [53]:
df_merged = df_smaller.merge(
    df_stats.add_prefix("home_"),
    left_on=["homeTeam.id", "matchday"],
    right_on=["home_team_id", "home_matchday"],
    how="left"
)

df_merged = df_merged.merge(
    df_stats.add_prefix("away_"),
    left_on=["awayTeam.id", "matchday"],
    right_on=["away_team_id", "away_matchday"],
    how="left"
)


In [81]:
df_merged = df_smaller\
    .merge(
        df_stats.add_prefix("home_"),
        left_on=["homeTeam.id", "matchday"],
        right_on=["home_team_id", "home_matchday"],
        how="left"
    )\
    .merge(
        df_stats.add_prefix("away_"),
        left_on=["awayTeam.id", "matchday"],
        right_on=["away_team_id", "away_matchday"],
        how="left"
    )


In [82]:
df_merged.drop(columns=["status","homeTeam.id","homeTeam.name","awayTeam.id","awayTeam.name",
    "home_team_id", "home_matchday", 
    "away_team_id", "away_matchday","score.fullTime.home","score.fullTime.away",
    "home_away_win_rate","home_away_loss_rate","away_home_win_rate","away_home_loss_rate",
    "home_cum_goal_diff","away_cum_goal_diff","home_cum_goals_for","away_cum_goals_for",
    "home_cum_goals_against","away_cum_goals_against"
],inplace=True)


In [85]:
df_for_training=df_merged[df_merged["matchday"]>5].drop(columns="matchday")

In [86]:
df_for_training.head()

,score.winner,home_points_before_match,home_avg_goals_for,home_avg_goals_against,home_home_win_rate,home_home_loss_rate,home_ranking_before_match,home_form_last5,away_points_before_match,away_avg_goals_for,away_avg_goals_against,away_away_win_rate,away_away_loss_rate,away_ranking_before_match,away_form_last5
45,HOME_TEAM,3.0,1.0,2.4,0.500000,0.5,11,3.0,6.0,1.6,2.0,0.000000,1.000000,11,6.0
46,HOME_TEAM,13.0,3.4,0.8,1.000000,0.0,12,13.0,7.0,1.8,1.2,0.000000,1.000000,12,7.0
47,DRAW,9.0,1.0,0.4,0.500000,0.0,16,9.0,7.0,2.8,1.2,0.333333,0.666667,16,7.0
48,AWAY_TEAM,6.0,1.4,2.0,0.500000,0.5,14,6.0,7.0,1.6,1.4,0.500000,0.500000,14,7.0
49,HOME_TEAM,13.0,2.0,0.4,0.666667,0.0,17,13.0,4.0,1.0,3.0,0.000000,1.000000,17,4.0
